In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append('..')

In [2]:
from utils.data_loading import load_users_data

train_users, test_users = load_users_data()

train_users = train_users.drop(['country_destination'], axis=1)
id_test = test_users['id']

users = pd.concat((train_users, test_users), axis=0, ignore_index=True)
users = users.drop(['id', 'date_first_booking'], axis=1)

users['gender'].replace('-unknown-', np.nan, inplace=True)
users['language'].replace('-unknown-', np.nan, inplace=True)

In [3]:
import datetime
users['date_account_created'] = pd.to_datetime(users['date_account_created'])
users['year_account_created'] = pd.DatetimeIndex(users['date_account_created']).year
users['month_account_created'] = pd.DatetimeIndex(users['date_account_created']).month
users['day_account_created'] = pd.DatetimeIndex(users['date_account_created']).day
users = users.drop(['date_account_created'], axis=1)

users['timestamp_first_active'] = pd.to_datetime(users['timestamp_first_active'], format='%Y%m%d%H%M%S')
users['year_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).year
users['month_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).month
users['day_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).day
users = users.drop(['timestamp_first_active'], axis=1)

In [4]:
from utils.preprocessing import one_hot_encoding

categorical_features = [
    'gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel',
    'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
    'first_device_type', 'first_browser'
]

users = one_hot_encoding(users, categorical_features)

In [5]:
# Age
age_values = users.age.values
users['age'] = np.where(np.logical_or(age_values < 14, age_values > 100), np.nan, age_values)

In [6]:
null_values = users['age'].isnull()

In [7]:
X = users.columns.tolist()
X.remove('age')

In [8]:
from sklearn.preprocessing import StandardScaler
users[X] = StandardScaler().fit_transform(users[X])

In [9]:
processed_users = users

In [10]:
users_with_age = users[~null_values]

In [12]:
from sklearn.svm import SVR
from sklearn import grid_search

parameters = {'kernel':['linear'], 'C':[1]}

svr = SVR()
clf = grid_search.GridSearchCV(svr, parameters, n_jobs=-1, scoring='r2', cv=5)

In [ ]:
clf.fit(
    users_with_age.drop('age', axis=1),
    users_with_age['age']
)

In [19]:
print clf.best_score_, 'with', clf.best_params_

<function sklearn.metrics.scorer._passthrough_scorer>